In [22]:
import numpy as np
import pandas as pd


class GradientBoostingRegressor:
    """Gradient boosting regressor."""
    def __init__(self):
        self.base_pred_ = None

    def fit(self, X, y):
        """Fit the model to the data.

        Args:
            X: array-like of shape (n_samples, n_features)
            y: array-like of shape (n_samples,)

        Returns:
            GradientBoostingRegressor: The fitted model.
        """
        self.base_pred_ = np.mean(y)
        return self
        
    

    def predict(self, X):
        """Predict the target of new data.

        Args:
            X: array-like of shape (n_samples, n_features)

        Returns:
            y: array-like of shape (n_samples,)
            The predict values.
            
        """
        # YOUR CODE HERE...
        predictions = np.full(X.shape[0], self.base_pred_)

        return predictions
    def _predict_one_sample(self, sample):
        return
        


In [23]:
data = pd.read_csv('data.csv')

In [24]:
X = data.values[:,-1]
y = data.values[:, -1]

In [25]:
gb = GradientBoostingRegressor()

In [26]:
gb.fit(X, y)

In [27]:
gb.predict(X)

array([13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
       13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791, 13.791,
      

In [53]:
import numpy as np
from typing import Tuple


def mse(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, np.ndarray]:
    """Mean squared error loss function and gradient."""

    loss = np.mean((y_pred - y_true) ** 2)
    grad = -2 * (y_pred - y_true)
    return loss, grad

def mae(y_true: np.ndarray, y_pred: np.ndarray) -> Tuple[float, np.ndarray]:
    """Mean absolute error loss function and gradient."""

    loss = np.mean(np.abs((y_pred - y_true)))
    grad = np.sign((y_pred - y_true))
    return loss, grad

In [54]:
y_true=np.array([1, 2, 3])
y_pred=np.array([5, 2, 1])

In [55]:
mse(y_true, y_pred)

(6.666666666666667, array([-8,  0,  4]))